<a href="https://colab.research.google.com/github/carlogattuso/Indoor_Positioning_RTT_GPR/blob/master/psfm_gpr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################ Library imports ################
import numpy as np
import math as math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import scipy.linalg
import seaborn as sns
import time
import glob
import pandas as pd

import itertools

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

from sklearn.metrics import mean_squared_error, mean_absolute_error
 
from scipy.optimize import curve_fit

from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
File reading
"""
df = pd.read_csv("https://drive.google.com/file/d/19gwdO1uiBRxF73U4ymTpy2T7VsCx6U65/view?usp=sharing", sep = ",", index_col=False)

ParserError: ignored

In [ ]:
"""
Dataset split
    - ap_obs --> RTT observations between APs
    - vrp_meas --> M 
"""
ap_obs = df.loc[df['x'].isin(L_ap[:,0]) & df['y'].isin(L_ap[:,1])]
print(ap_obs.shape)
vrp_meas = df.loc[df['x'].isin(L_vrp[:,0]) & df['y'].isin(L_vrp[:,1])]
print(ap_obs.shape)

In [ ]:
print(ap_obs.loc[(df['x'] == 0) & (df['y'] == 0)].shape)
print(ap_obs.loc[(df['x'] == 4) & (df['y'] == 0)].shape)
print(ap_obs.loc[(df['x'] == 8) & (df['y'] == 0)].shape)
print(ap_obs.loc[(df['x'] == 0) & (df['y'] == 9.6)].shape)
print(ap_obs.loc[(df['x'] == 0) & (df['y'] == 19.2)].shape)
print(ap_obs.loc[(df['x'] == 4) & (df['y'] == 19.2)].shape)
print(ap_obs.loc[(df['x'] == 8) & (df['y'] == 19.2)].shape)

In [ ]:
print(L_ap[:,0])
print(L_ap[:,1])
print(L_ap[:,0].shape)
print(L_ap[:,1].shape)
print(L_vrp[:,0])
print(L_vrp[:,1])
print(L_vrp[:,0].shape)
print(L_vrp[:,1].shape)

In [ ]:
ap_obs = df.loc[df['x'].isin(L_ap[:,0])]
ap_obs = ap_obs.loc[df['y'].isin(L_ap[:,1])]
print(ap_obs.shape)

In [ ]:
"""
Scenario definition
"""
n_ap = 7

height = 8
width = 19.2

ap_x = 5
ap_y = 5

locations = np.array(np.meshgrid(np.linspace(0, width, ap_x).round(2), np.linspace(0, height, ap_y).round(2))).T.reshape(-1,2)
locations[:, [1, 0]] = locations[:, [0, 1]]

# ----------------------------------------------------------------------
#  AP locations
L_ap = locations[[0,2,4,10,20,22,24],:]
print(L_ap)
# ----------------------------------------------------------------------
#  VRP locations
L_vrp = np.delete(locations, [0,2,4,10,20,22,24], 0)

# ----------------------------------------------------------------------
#  AP Labels & Indexes
ap_labels = { '0':'Google Blanco' , '1':'Linksys 1' , '2':'Google Azul' , '3':'Linksys 2' , '4':'Google Rojo' , '5':'Linksys 3',
'6':'Google Verde' }
ap_indexes = { 'Google Blanco':'0' , 'Linksys 1':'1' , 'Google Azul':'2' , 'Linksys 2':'3' , 'Google Rojo':'4' , 'Linksys 3':'5',
'Google Verde':'6' }

In [ ]:
"""
Plot coordinates
"""
coordinates = np.array(np.meshgrid(np.linspace(0, 4, 5), np.linspace(0, 4, 5))).T.reshape(-1,2)
coordinates[:, [1, 0]] = coordinates[:, [0, 1]]

coordinates_ap = coordinates[[0,2,4,10,20,22,24],:]

coordinates_vrp = np.delete(coordinates, [0,2,4,10,20,22,24], 0)

In [ ]:
"""
Locations plot
"""

plt.figure(figsize=(8,6))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-5)
plt.title('Scenario',pad=20)
plt.axis('off')

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 6, 7).astype(int)):
    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

In [ ]:
"""
Compute Euclidean distance between APs
    
    array --> Array of AP/VRP locations
"""
def compute_euclidean_distance(array):
    n = array.shape[0]

    d_mat = [np.linalg.norm(i - j) for (i,j) in itertools.product(array, array)]
    d_mat = np.array(d_mat).reshape((n, n), order='F')

    return d_mat

In [ ]:
"""
Target AP to compute algorithm: AP[0,0]

Let's assume an euclidean distance as RTT distance observations
""" 
d_RTT = compute_euclidean_distance(L_ap)

##Select one AP to plot RTT distance observations

In [ ]:
menu = widgets.Dropdown(
       options=['Google Blanco', 'Linksys 1', 'Google Azul', 'Linksys 2', 'Google Rojo', 'Linksys 3', 'Google Verde'],
       value='Google Blanco',
       description='AP:')
menu

In [ ]:
plt.figure(figsize=(8,4))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')

plt.legend(labels=['AP'],loc="upper right",borderaxespad=-5)
plt.title('RTT observations between target AP '+menu.value+' and the others',pad=30)
plt.axis('off')

for x,y,z in zip(coordinates_ap[:,0], coordinates_ap[:,1], d_RTT[:,int(ap_indexes[menu.value])]):
    label = f"{z.round(2)} m"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 6, 7).astype(int)):
    label = f"({L_ap[z,0]},{L_ap[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-40), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

In [ ]:
"""
Polynomial Surface model fitting
    
    loc --> X,Y tuple array of AP locations
    d --> AP RTT distance observations
    order --> Polinomial order
"""
def surface_fit(loc, d, order=1):
    if order == 1:
        A = np.c_[loc[:,0], loc[:,1], np.ones(loc.shape[0])]
        C,_,_,_ = scipy.linalg.lstsq(A, d)
    else:
        A = np.c_[np.ones(loc.shape[0]), loc[:,:2], np.prod(loc[:,:2], axis=1), loc[:,:2]**2]
        C,_,_,_ = scipy.linalg.lstsq(A, d)
    return C

In [ ]:
"""
RTT distance estimation
    
    x --> AP's x locations
    y --> AP's y locations
    C --> Model coefficients
"""
def sample_distance(x, y, C):
    
    XX = x.flatten()
    YY = y.flatten()
    
    if C.shape[0] == 3:
        fit = np.dot(np.c_[XX, YY, np.ones(XX.shape)], C).reshape(x.shape)
    else:
        fit = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX*YY, XX**2, YY**2], C).reshape(x.shape)
    return fit

In [ ]:
"""
Surface fitting models
"""

# Plot domain
X,Y = np.meshgrid(np.arange(0.0, 10, 2), np.arange(0.0, 24, 4.8))

#Models fitting and surface estimation
models_1 = np.array([surface_fit(L_ap, d_RTT[:,i]) for (i) in np.linspace(0, 6, 7).astype(int)])
models_2 = np.array([surface_fit(L_ap, d_RTT[:,i],2) for (i) in np.linspace(0, 6, 7).astype(int)])

Z_1 = np.array([sample_distance(X, Y, models_1[i]) for (i) in np.linspace(0, 6, 7).astype(int)])
Z_2 = np.array([sample_distance(X, Y, models_2[i]) for (i) in np.linspace(0, 6, 7).astype(int)])

##Select one AP to plot fitting model

In [ ]:
menu_2 = widgets.Dropdown(
       options=['Google Blanco', 'Linksys 1', 'Google Azul', 'Linksys 2', 'Google Rojo', 'Linksys 3', 'Google Verde'],
       value='Google Blanco',
       description='AP:')
menu_2

In [ ]:
"""
Surface fitting plot
"""
fig = plt.figure(figsize=(12,8))

ax = fig.gca(projection='3d')

l1 = ax.plot_surface(X, Y, Z_1[int(ap_indexes[menu_2.value]),:,:], rstride=1, cstride=1, alpha=0.5, label='linear', color='g')
l2 = ax.plot_surface(X, Y, Z_2[int(ap_indexes[menu_2.value]),:,:], rstride=1, cstride=1, alpha=0.5, label='quadratic', color='b')

ax.scatter(L_ap[:,0], L_ap[:,1], d_RTT[:,int(ap_indexes[menu_2.value])], c='r', s=50)

plt.xlabel('X')
plt.ylabel('Y')
ax.set_zlabel('m')
ax.axis('auto')

line_labels = ["AP locations","Linear", "Quadratic"]

ax.plot([0], [0], d_RTT[:,int(ap_indexes[menu_2.value])][0], color="red", marker='o', linewidth=0)[0]
ax.plot([0], [0], color="green")[0]
ax.plot([0], [0], color="blue")[0]

fig.legend(labels=line_labels,loc="upper right",borderaxespad=4)

plt.title('AP RTT distance models')
plt.show()

In [ ]:
"""
Mean RTT distance values at VRPs
    _1 --> Order 1 surface fit
    _2 --> Order 2 surface fit
"""

fit_1 = np.reshape(Z_1, (n_ap, 25, 1))
fit_2 = np.reshape(Z_2, (n_ap, 25, 1))

# ----------------------------------------------------------------------
#  Mean RTT observations between target AP and the rest
m_ap_1 = fit_1[:,[0,2,4,10,20,22,24],:]
m_ap_2 = fit_2[:,[0,2,4,10,20,22,24], :]
vrp_pts = np.delete(np.linspace(0, 24, 25).astype(int), [0,2,4,10,20,22,24], 0)

# ----------------------------------------------------------------------
#  Mean RTT observations between target AP and VRPs
m_vrp_1 = fit_1[:,vrp_pts,:]
m_vrp_2 = fit_2[:,vrp_pts,:]

##Select one AP to plot RTT Mean distances

In [ ]:
menu_3 = widgets.Dropdown(
       options=['Google Blanco', 'Linksys 1', 'Google Azul', 'Linksys 2', 'Google Rojo', 'Linksys 3', 'Google Verde'],
       value='Google Blanco',
       description='AP:')
menu_3

In [ ]:
"""
Order 1 RTT Mean distance plot
"""
plt.figure(figsize=(8,6))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('AP '+menu_3.value+': Order 1 Fitting mean RTT distance values',pad=60)
plt.axis('off')

for x,y,z in zip(coordinates[:,0], coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"{sample_distance(locations[z,0], locations[z,1], models_1[int(ap_indexes[menu_3.value])]).round(2)} m"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

In [ ]:
"""
Order 2 RTT Mean distance plot
"""
plt.figure(figsize=(8,6))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('AP '+menu_3.value+': Order 2 Fitting mean RTT distance values',pad=60)
plt.axis('off')

for x,y,z in zip(coordinates[:,0], coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"{sample_distance(locations[z,0], locations[z,1], models_2[int(ap_indexes[menu_3.value])]).round(2)} m"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

In [ ]:
"""
Gaussian Process Kernel definition

Kernel parameters:
    - L --> Length Scale
    - sigma_f --> Constant Kernel Amplitude
"""

def kernel_function(x, y, sigma_f=1, l=1):

    """ Squared Exponential kernel function """
    kernel = sigma_f * np.exp(- (np.linalg.norm(x - y)**2) / (2 * l**2))
    return kernel

In [ ]:
"""
Hyperparameter optimization of sigma_f and length-scale

    loc --> AP locations
    mean --> RTT distance observations between APs
"""
def gp_fit(loc, mean):
    
    gp.fit(loc, mean)

    l = gp.kernel_.get_params().get('k2__length_scale')
    sigma_f = gp.kernel_.get_params().get('k1__constant_value')
    return l, sigma_f

In [ ]:
"""
Gaussian Process Fit using scikit-learn GPR
"""
sigma_n = 0.0

kernel = ConstantKernel(constant_value=100.0,constant_value_bounds=(1e-3, 1e3)) \
            * RBF(length_scale=10.0, length_scale_bounds=(1e-3, 1e3))

gp = GaussianProcessRegressor(kernel=kernel, alpha=sigma_n**2, n_restarts_optimizer=0)

hyper_params = np.array([gp_fit(L_ap, d_RTT[:,i]) for (i) in np.linspace(0, 6, 7).astype(int)])
print(hyper_params[1])

In [ ]:
def compute_cov_matrices(L_ap, L_vrp, sigma_f=1, l=1):
    """
    Compute components of the covariance matrix of the joint distribution.
    
    We follow the notation:
    
        - K = K(L, L) 
        - K_star = K(L^*, L)
        - K_star2 = K(L^*, L^*)
    """
    n = L_ap.shape[0]
    n_star = L_vrp.shape[0]

    K = [kernel_function(i, j, sigma_f=sigma_f, l=l) for (i, j) in itertools.product(L_ap, L_ap)]

    K = np.array(K).reshape(n, n)
    
    K_star2 = [kernel_function(i, j, sigma_f=sigma_f, l=l) for (i, j) in itertools.product(L_vrp, L_vrp)]

    K_star2 = np.array(K_star2).reshape(n_star, n_star)
    
    K_star = [kernel_function(i, j, sigma_f=sigma_f, l=l) for (i, j) in itertools.product(L_vrp, L_ap)]

    K_star = np.array(K_star).reshape(n_star, n)
    
    return (K, K_star2, K_star)

In [ ]:
class Regressor:
    '''
    Handles the required structures to compute GPR interpolation

            Parameters:
                    K (ndarray): Covariance matrix between AP locations
                    K_star2 (ndarray): Covariance matrix between VRP locations
                    K_star (ndarray): Covariance matrix between each AP location and each VRP location
                    m_ap (ndarray): Mean RTT distance values at AP locations from fitting model
                    m_vrp (ndarray): Mean RTT distance values at VRP locations from fitting model
    '''
    def __init__(self, _K, _K_star2, _K_star, _m_ap, _m_vrp, _m_ap2, _m_vrp2):
        self.K = _K
        self.K_star2 = _K_star2
        self.K_star = _K_star
        self.m_ap = _m_ap
        self.m_vrp = _m_vrp
        self.m_ap2 = _m_ap2
        self.m_vrp2 = _m_vrp2


In [ ]:
"""
Gaussian Process Regressor definition:

    - Covariance matrices: K, K_star, K_star2
    - Mean distance values: m_ap, m_vrp 
"""

list = []

for i in np.linspace(0, 6, 7).astype(int):
    a, b, c = compute_cov_matrices(L_ap, L_vrp, sigma_f=hyper_params[i][1], l=hyper_params[i][0])
    gpr = Regressor(a,b,c,m_ap_1[i],m_vrp_1[i],m_ap_2[i],m_vrp_2[i])
    list.append(gpr)

##Select one AP to view Covariance Matrices

In [ ]:
menu_4 = widgets.Dropdown(
       options=['Google Blanco', 'Linksys 1', 'Google Azul', 'Linksys 2', 'Google Rojo', 'Linksys 3', 'Google Verde'],
       value='Google Blanco',
       description='AP:')
menu_4

In [ ]:
"""
Multivariate Joint Distribution: G matrix
"""

n = L_ap.shape[0]
n_star = L_vrp.shape[0]

a = np.concatenate((list[int(ap_indexes[menu_4.value])].K + (sigma_n**2)*np.eye(n), list[int(ap_indexes[menu_4.value])].K_star), axis=0)
print(a.shape)
b = np.concatenate((list[int(ap_indexes[menu_4.value])].K_star.T, list[int(ap_indexes[menu_4.value])].K_star2), axis=0)
print(b.shape)
G = np.concatenate((a, b), axis=1)
print(G.shape)
G.shape == (n + n_star, n + n_star)

In [ ]:
"""
Kernel Covariance Matrix K(L,L)
"""
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(data=list[int(ap_indexes[menu_4.value])].K, cmap='Blues', ax=ax)
ax.set(title='AP '+menu_4.value+' : Components of the Kernel Covariance Matrix K(L,L)')

In [ ]:
"""
Kernel Covariance Matrix K(L*,L*)
"""
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(data=list[int(ap_indexes[menu_4.value])].K_star2, cmap='Blues', ax=ax)
ax.set(title='AP '+menu_4.value+' : Components of the Kernel Covariance Matrix K(L*,L*)')

In [ ]:
"""
Kernel Covariance Matrix K(L*,L)
"""
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(data=list[int(ap_indexes[menu_4.value])].K_star, cmap='Blues', ax=ax)
ax.set(title='AP '+menu_4.value+' : Components of the Kernel Covariance Matrix K(L*,L)')

In [ ]:
def psfm_gpr_parameters(m, m_star, d, K, K_star2, K_star, sigma_n):
    
    """Compute gaussian regression parameters."""

    n = K.shape[0]
    n_star = K_star.shape[0]
    
    # Mean.
    s_bar_star = (m_star.reshape([n_star, 1]) 
        + np.dot(K_star, np.dot(np.linalg.inv(K + (sigma_n**2)*np.eye(n)), d.reshape([n, 1]) - m.reshape([n, 1]))))
    # Covariance.
    cov_s_star = K_star2 - np.dot(K_star, np.dot(np.linalg.inv(K + (sigma_n**2)*np.eye(n)), K_star.T))
    
    return (s_bar_star.reshape(n_star), cov_s_star)

In [ ]:
def zero_mean_gpr_parameters(d, K, K_star2, K_star, sigma_n):
    
    """Compute gaussian regression parameters."""

    n = K.shape[0]
    n_star = K_star.shape[0]

    # Mean.
    s_bar_star = np.dot(K_star, np.dot(np.linalg.inv(K + (sigma_n**2)*np.eye(n)), d.reshape([n, 1])))
    # Covariance.
    cov_s_star = K_star2 - np.dot(K_star, np.dot(np.linalg.inv(K + (sigma_n**2)*np.eye(n)), K_star.T))
    
    return (s_bar_star.reshape(n_star), cov_s_star)

In [ ]:
"""
Zero-Mean Gaussian Process interpolations
"""
n = L_ap.shape[0]
n_star = L_vrp.shape[0]

means = np.empty((n, n_star, 1))
variances = np.empty((n, n_star, 1))

for i in np.linspace(0, 6, 7).astype(int):

    s_bar_star, cov_s_star = zero_mean_gpr_parameters(d_RTT[:,i], list[i].K, list[i].K_star2, list[i].K_star, sigma_n)
    var_s_star = np.diag(cov_s_star)

    means[i,:,:] = s_bar_star.reshape(n_star,1)
    variances[i,:,:] = var_s_star.reshape(n_star,1)

In [ ]:
"""
Order 1 PSFM Gaussian Process interpolations
"""
means_psfm = np.empty((n, n_star, 1))

for i in np.linspace(0, 6, 7).astype(int):
    s_bar_star, cov_s_star = psfm_gpr_parameters(list[i].m_ap,list[i].m_vrp, d_RTT[:,i], list[i].K, list[i].K_star2, list[i].K_star, sigma_n)
    means_psfm[i,:,:] = s_bar_star.reshape(n_star,1)

In [ ]:
"""
Order 2 PSFM Gaussian Process interpolations
"""
means_psfm2 = np.empty((n, n_star, 1))

for i in np.linspace(0, 6, 7).astype(int):
    s_bar_star, cov_s_star = psfm_gpr_parameters(list[i].m_ap2,list[i].m_vrp2, d_RTT[:,i], list[i].K, list[i].K_star2, list[i].K_star, sigma_n)
    means_psfm2[i,:,:] = s_bar_star.reshape(n_star,1)

##Select one AP to plot Gaussian Process Interpolations

In [ ]:
menu_5 = widgets.Dropdown(
       options=['Google Blanco', 'Linksys 1', 'Google Azul', 'Linksys 2', 'Google Rojo', 'Linksys 3', 'Google Verde'],
       value='Google Blanco',
       description='AP:')
menu_5

In [ ]:
"""
Zero Mean Gaussian Process interpolation:
    
    s_bar_star --> Mean estimated value for each VRP
    cov_s_star --> Posterior covariance matrix
    var_s_star --> VRP estimation variance
"""

plt.figure(figsize=(9,6))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('AP '+menu_5.value+' : Zero Mean Gaussian Process interpolation',pad=60)
plt.axis('off')

for x,y,z in zip(coordinates_ap[:,0], coordinates_ap[:,1], d_RTT[:,int(ap_indexes[menu_5.value])]):
    label = f"{z.round(2)} m"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x in range(len(means[int(ap_indexes[menu_5.value])])):
    label = f"{means[int(ap_indexes[menu_5.value])][x,0].round(5)} +- {variances[int(ap_indexes[menu_5.value])][x,0].round(3)} m"

    plt.annotate(label, # this is the text
                 (coordinates_vrp[x,0],coordinates_vrp[x,1]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

In [ ]:
"""
First Order PSFM Gaussian Process interpolation:
    
    s_bar_star --> Mean estimated value for each VRP
    cov_s_star --> Posterior covariance matrix
    var_s_star --> VRP estimation variance
"""

plt.figure(figsize=(9,6))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('AP '+menu_5.value+' : First Order PSFM Gaussian Process interpolation',pad=60)
plt.axis('off')

for x,y,z in zip(coordinates_ap[:,0], coordinates_ap[:,1], d_RTT[:,int(ap_indexes[menu_5.value])]):
    label = f"{z.round(2)} m"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x in range(len(means_psfm[int(ap_indexes[menu_5.value])])):
    label = f"{means_psfm[int(ap_indexes[menu_5.value])][x,0].round(5)} +- {variances[int(ap_indexes[menu_5.value])][x,0].round(3)} m"

    plt.annotate(label, # this is the text
                 (coordinates_vrp[x,0],coordinates_vrp[x,1]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

In [ ]:
"""
Second Order PSFM Gaussian Process interpolation:
    
    s_bar_star --> Mean estimated value for each VRP
    cov_s_star --> Posterior covariance matrix
    var_s_star --> VRP estimation variance
"""

plt.figure(figsize=(9,6))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('AP '+menu_5.value+' : Second Order PSFM Gaussian Process interpolation',pad=60)
plt.axis('off')

for x,y,z in zip(coordinates_ap[:,0], coordinates_ap[:,1], d_RTT[:,int(ap_indexes[menu_5.value])]):
    label = f"{z.round(2)} m"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x in range(len(means_psfm2[int(ap_indexes[menu_5.value])])):
    label = f"{means_psfm2[int(ap_indexes[menu_5.value])][x,0].round(5)} +- {variances[int(ap_indexes[menu_5.value])][x,0].round(3)} m"

    plt.annotate(label, # this is the text
                 (coordinates_vrp[x,0],coordinates_vrp[x,1]), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,-20), # distance from text to points (x,y)
                 ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

In [ ]:
"""
Zero Mean GPR Fingerprinting Database plot
"""

plt.figure(figsize=(12,16))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('Zero Mean GPR Fingerprinting Database plot',pad=60)
plt.axis('off')

for x in range(len(d_RTT[:,0])):
    for y in range(len(d_RTT[:,0])):
        label = f"AP {y}: {d_RTT[x,y].round(3)} m"

        plt.annotate(label, # this is the text
                    (coordinates_ap[x,0],coordinates_ap[x,1]), # this is the point to label
                    textcoords="offset points", # how to position the text
                    xytext=(0,-20-y*20), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center

for x in range(len(means[0])):
    for y in range(len(d_RTT[:,0])):
        label = f"AP {y}: {means[:,x,:][y,0].round(3)} +- {variances[:,x,:][y,0].round(3)} m"

        plt.annotate(label, # this is the text
                    (coordinates_vrp[x,0],coordinates_vrp[x,1]), # this is the point to label
                    textcoords="offset points", # how to position the text
                    xytext=(0,-20-y*20), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center


for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')


for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

In [ ]:
"""
Order 1 PSFM GPR Fingerprinting Database plot
"""

plt.figure(figsize=(12,16))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('Order 1 PSFM GPR Fingerprinting Database plot',pad=60)
plt.axis('off')

for x in range(len(d_RTT[:,0])):
    for y in range(len(d_RTT[:,0])):
        label = f"AP {y}: {d_RTT[x,y].round(3)} m"

        plt.annotate(label, # this is the text
                    (coordinates_ap[x,0],coordinates_ap[x,1]), # this is the point to label
                    textcoords="offset points", # how to position the text
                    xytext=(0,-20-y*20), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center

for x in range(len(means_psfm[0])):
    for y in range(len(d_RTT[:,0])):
        label = f"AP {y}: {means_psfm[:,x,:][y,0].round(3)} +- {variances[:,x,:][y,0].round(3)} m"

        plt.annotate(label, # this is the text
                    (coordinates_vrp[x,0],coordinates_vrp[x,1]), # this is the point to label
                    textcoords="offset points", # how to position the text
                    xytext=(0,-20-y*20), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center

In [ ]:
"""
Order 2 PSFM GPR Fingerprinting Database plot
"""

plt.figure(figsize=(12,16))

plt.scatter(coordinates_ap[:,0], coordinates_ap[:,1], c='b')
plt.scatter(coordinates_vrp[:,0], coordinates_vrp[:,1], c='r')

plt.legend(labels=['AP','VRP'],loc="upper right",borderaxespad=-8)
plt.title('Order 2 PSFM GPR Fingerprinting Database plot',pad=60)
plt.axis('off')

for x in range(len(d_RTT[:,0])):
    for y in range(len(d_RTT[:,0])):
        label = f"AP {y}: {d_RTT[x,y].round(3)} m"

        plt.annotate(label, # this is the text
                    (coordinates_ap[x,0],coordinates_ap[x,1]), # this is the point to label
                    textcoords="offset points", # how to position the text
                    xytext=(0,-20-y*20), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center

for x in range(len(means_psfm[0])):
    for y in range(len(d_RTT[:,0])):
        label = f"AP {y}: {means_psfm[:,x,:][y,0].round(3)} +- {variances[:,x,:][y,0].round(3)} m"

        plt.annotate(label, # this is the text
                    (coordinates_vrp[x,0],coordinates_vrp[x,1]), # this is the point to label
                    textcoords="offset points", # how to position the text
                    xytext=(0,-20-y*20), # distance from text to points (x,y)
                    ha='center') # horizontal alignment can be left, right or center

for x,y,z in zip(coordinates_ap[:,0],coordinates_ap[:,1], np.linspace(0, 7, 8).astype(int)):

    label = f"{ap_labels[str(z)]}"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,30), # distance from text to points (x,y)
                 ha='center', # horizontal alignment can be left, right or center
                 weight='bold')

for x,y,z in zip(coordinates[:,0],coordinates[:,1], np.linspace(0, 24, 25).astype(int)):
    label = f"({locations[z,0]},{locations[z,1]})"

    plt.annotate(label, # this is the text
                 (x,y), # this is the point to label
                 textcoords="offset points", # how to position the text
                 xytext=(0,10), # distance from text to points (x,y)
                 ha='center',
                 weight='bold') # horizontal alignment can be left, right or center